# PREN2
Path to recognize these little icons
## Data Preparation for the training
First we need to prepare image data and icons, therefore they must fit to the desired training. As Base_path we use `training` and all source files goes into `training/source` directory. F.E. the targetpictograms.
### Download Targetpictograms
Convert and crop them with *imagemagick*
```console
convert -crop 1000x1000-10-50 -gravity center +repage -density 150 Piktogramme_PREN1_HS20.pdf -quality 100 -resize 200x200 output-img.jpg
```

## Negative Samples
Negative Samples sind beliebige Bilder, welche das Objekt nicht enthalten dürfen. 
1. Download Negative Bilder https://github.com/sonots/tutorial-haartraining/tree/master/data/negatives
1. in Arbeitsverzeichnis `training/negatives` ablegen
1. TXT-Dokument mit Pfad zu allen negativen Bilder erstellen: `find negatives -type f > negatives.txt`

## Create Positive Samples
1. `cd sources`
1. For each icon run `create_positive_variants.sh <iconname>` as parameter pass the **name** image whithout fileextension. If you have another directory structure, modify the constants in the script. probably you have to set the permissions to run it `chmod a+rx create_positive_variants.sh` It creates ten variants of each icon
1. For each icon run `echo_create_positive_commands.sh <iconname>` with the image name as parameter and run it. check permissions either and run it. This script echoes the commands to create positive images of those variants created in previous step. copy and run each of them.
1. add path as prefix to all lines in samplefile created by script (for each, use bbedit or textwrangler to achieve)
1. run `create_negatives_samples.sh <iconname>` to create a set of negative images, containing other icons. adjust the array `IMG_TO_COMPOSITE` in script **before** running
1. run script `echo_prep-training_commands.sh <iconname>` to echoes commands for further processing. you have to
    1. collect all sample file into one
    1. collect all negative samples include the positive samples containing other icons
    1. create vec file
1. Server schicken `scp -r /Users/stofers/Development/HSLU/PREN/pren_stair_climb/Raspberry_Pi/pictograms/training localadm@prenh20-sstofer.el.eee.intern:~/training`
1. opencv_trainingcascade must be in same directory as the data! `cp ~/build/bin/opencv_traincascade training/` if not exists
1. run training with `./opencv_traincascade -data hammer -vec hammer.vec -bg hammer_negatives.txt -numPos 13500 -numNeg 8280 -numStages 20 -w 48 -h 48 > hammer/log_hammer_training.txt`
1. Copy cascade.xml to local directory `scp localadm@prenh20-sstofer.el.eee.intern:~/training/hammer/cascade.xml hammer/cascade.xml`

## Verification

In [1]:
import cv2

classifier = "hammer"
item_clsfr=cv2.CascadeClassifier("../"+classifier+"/cascade.xml")
#loading the cascade classifier

camera=cv2.VideoCapture(0)
#initializing the video object (0 for default webcam)

while(True):
#infinite loop to read continuous frames from the camera object

    ret,img=camera.read()
    #reading a single frame from the camera
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #converting the color image to a gray scale image
    items=item_clsfr.detectMultiScale(gray)     
    #detecting in the gray scale
    #hammers is a 2D array contaning n number of rows (n= number of hammers in the frame), 4 columns (x,y,w,h)
    for (x,y,w,h) in items:
    #going through each and assigning the x,y,w,h

        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        #Drawing a rectangle bounding the faces
        cv2.putText(img,classifier,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)
            
    cv2.imshow('LIVE',img)
    cv2.waitKey(1)
    #showing the frame

KeyboardInterrupt: 

## Some basic stuff

command create positive samples based on input images and a text-file pointing to negative images. the input image will randomly (see parameters) placed on those and position is annotated in a text file.
```console
/Users/stofers/openCV/opencv/cmake-build-debug/bin/opencv_createsamples -info /Users/stofers/Development/HSLU/PREN/pren_stair_climb/Raspberry_Pi/pictograms/training/hammer/hammer.txt -bg /Users/stofers/Development/HSLU/PREN/pren_stair_climb/Raspberry_Pi/pictograms/training/negatives_all.txt -img /Users/stofers/Development/HSLU/PREN/pren_stair_climb/Raspberry_Pi/pictograms/training/sources/hammer.jpg -num 1500 -w 24 -h 24 -maxxangle 0.2 -maxzangle 0.2 -maxyangle 0.2 -bgcolor 100
```
command *prints* filecontent and put it into a new document
```console
`cat FILENAME*.txt > positives.txt`
````

command list filenames ending mit .txt in the directory hammer and list them in a new file
```console
find hammer -name "*.txt" > hammer/hammer_samples.txt
```

command creates a vec-file, which is needed for cascade training.
```console
/Users/stofers/openCV/opencv/cmake-build-debug/bin/opencv_createsamples -info /Users/stofers/Development/HSLU/PREN/pren_stair_climb/Raspberry_Pi/pictograms/training/hammer/hammer_samples.txt -bg /Users/stofers/Development/HSLU/PREN/pren_stair_climb/Raspberry_Pi/pictograms/training/hammer_negatives.txt -vec /Users/stofers/Development/HSLU/PREN/pren_stair_climb/Raspberry_Pi/pictograms/training/hammer/hammer.vec -num 15000 -w 48 -h 48
```